In [5]:
import pandas as pd
import numpy as np
from azure.storage.blob import BlobServiceClient
import io
import pyarrow.parquet as pq
import pyarrow as pa
import re
import os

# Credenciais
account_name = 'uvabr'
account_key = os.environ['ACCOUNT_KEY']
container_name_read = 'bronze'
container_name_send = 'silver'

endpoints = [
    "deputadosDespesas",
#    "proposicoes",
#    "proposicoesTemas",
#    "proposicoesAutores",
#    "frentes",
#    "frentesDeputados",
#    "grupos",
#    "gruposMembros",
#    "gruposHistorico",
#    "legislaturas",
#    "legislaturasMesas",
#    "orgaos",
#    "orgaosDeputados",
    "deputados",
#    "deputadosOcupacoes",
#    "deputadosProfissoes",
#    "eventos",
#    "eventosOrgaos",
#    "eventosRequerimentos",
#    "votacoes",
#    "votacoesOrientacoes",
#    "votacoesVotos",
#    "votacoesObjetos",
#    "votacoesProposicoes",
#    "funcionarios",
#    "licitacoes",
#    "licitacoesContratos",
#    "licitacoesItens",
#    "licitacoesPedidos",
#    "licitacoesPropostas",
#    "tecadTermos",
#    "tecadCategorias",
#    "deputadosCompleto",
#    "partidosCompleto"
]

def ler_parquet_azure(account_name, account_key, container_name_read, file_name):
    # Conecte-se ao serviço de armazenamento de blob
    connect_str = f"DefaultEndpointsProtocol=https;AccountName={account_name};AccountKey={account_key};EndpointSuffix=core.windows.net"
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)

    # Acesse o contêiner
    container_client = blob_service_client.get_container_client(container_name_read)

    # Baixe o blob do Azure Blob Storage
    blob_client = container_client.get_blob_client(file_name)
    blob_data = blob_client.download_blob()
    blob_stream = io.BytesIO()
    blob_stream.write(blob_data.readall())

    # Leia o arquivo Parquet usando pyarrow
    parquet_table = pq.read_table(blob_stream)

    # Converta para DataFrame do Pandas
    df = parquet_table.to_pandas()

    return df

def enviar_parquet_azure(account_name, account_key, container_name_send, file_name, dataframe):
    # Conecte-se ao serviço de armazenamento de blob
    connect_str = f"DefaultEndpointsProtocol=https;AccountName={account_name};AccountKey={account_key};EndpointSuffix=core.windows.net"
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)

    # Acesse o contêiner
    container_client = blob_service_client.get_container_client(container_name_send)

    # Converta DataFrame para tabela Parquet
    table = pa.Table.from_pandas(dataframe)

    # Crie um buffer de memória para escrever os dados Parquet
    parquet_buffer = io.BytesIO()
    pq.write_table(table, parquet_buffer)

    # Envie os dados para o Azure Blob Storage
    blob_client = container_client.get_blob_client(file_name)
    blob_client.upload_blob(parquet_buffer.getvalue(), overwrite=True)

    print(f"Arquivo Parquet '{file_name}' enviado com sucesso para o Azure Blob Storage.")

def criar_coluna_uri(dataframe, endpoint):
    # Padronize o prefixo da coluna com base no endpoint
    prefixo_coluna = "uri_" + endpoint

    # Para cada linha no DataFrame
    for indice, linha in dataframe.iterrows():
        # Para cada coluna no DataFrame
        for coluna, valor in linha.items():
            # Se a coluna contiver "uri" em algum lugar no nome
            if "uri" in coluna.lower():
                # Use expressão regular para extrair o ID do URI
                match = re.search(r'(\d+)$', str(valor))

                # Se houver uma correspondência na expressão regular
                if match:
                    # Crie uma nova coluna com o prefixo apropriado e atribua o valor extraído
                    nova_coluna = prefixo_coluna + "_" + coluna.replace("uri", "").strip("_")
                    # Remova o sublinhado no final do nome da coluna, se houver
                    nova_coluna = nova_coluna.rstrip('_')
                    dataframe.at[indice, nova_coluna] = match.group(1)
    return dataframe        

def transformar_coluna_data(dfdata):
    for coluna in dfdata.columns:
        if 'data' in coluna.lower():  # Verifica se o nome da coluna contém 'data'
            try:
                dfdata[coluna] = dfdata[coluna].apply(lambda x: '' if (x == None or x < ('1800-01-01') or x > ('2099-12-31') ) else x)
                dfdata[coluna] = pd.to_datetime(dfdata[coluna])  # Converte para datetime
            except ValueError:
                print(f"Erro ao converter coluna {coluna} para datetime.")
    return dfdata

def transform_id_cod_to_string(df):
    for coluna in df.columns:
        if 'id' in coluna.lower() or 'cod' in coluna.lower():
            df[coluna] = df[coluna].astype(str)
    return df

def transformar_valores(df):
    for column in df.columns:
        if 'url' not in column.lower():  # Verifica se "url" NÃO está no nome da coluna
            if df[column].dtype != 'datetime64[ns]' and df[column].dtype != 'float64':
                df[column] = df[column].replace(['', '<NA>', '[]', pd.NaT, np.nan, None], 'N/A')
            elif df[column].dtype == 'float64':
                print("nada funciona aqui")
                df[column] = df[column].fillna('N/A').astype('str')
    return df

def transformar_valores_float(df):
    for column in df.columns:
        if 'valor' in column.lower():
            print(column)# Verifica se "url" NÃO está no nome da coluna
            if df[column].dtype == 'object' or df[column].dtype == 'string':
                df[column] = df[column].astype(float)
                print("transformacao concluida com sucessso")  
            else:
                print("Deu ruim")             
    return df

def converter_float64_to_int(df):
    float_cols = df.select_dtypes(include=['float64']).columns
    df[float_cols] = df[float_cols].astype('int')
    return df


#tratamento direto

#tratamento global
def processar_endpoints(account_name, account_key, container_name_read, container_name_send, endpoints):
    for endpoint in endpoints:
        print(endpoint)
        df = ler_parquet_azure(account_name, account_key, container_name_read, f'{endpoint}.parquet')
        print(f"Leitura do {endpoint}.parquet concluída")
        df = criar_coluna_uri(df, endpoint)
        df = transformar_coluna_data(df)
        df = transform_id_cod_to_string(df)
        df = transformar_valores(df)
        df = transformar_valores_float(df)
#        df = converter_float64_to_int(df) 
        print(f"Tratamento do {endpoint}.parquet concluído")
        enviar_parquet_azure(account_name, account_key, container_name_send, f'{endpoint}.parquet', df)
        print(f"Envio do {endpoint}.parquet concluído")
        
# Usando a função para processar todos os endpoints
processar_endpoints(account_name, account_key, container_name_read, container_name_send, endpoints)

deputadosDespesas
Leitura do deputadosDespesas.parquet concluída
valorDocumento
transformacao concluida com sucessso
valorGlosa
transformacao concluida com sucessso
valorLiquido
transformacao concluida com sucessso
Tratamento do deputadosDespesas.parquet concluído
Arquivo Parquet 'deputadosDespesas.parquet' enviado com sucesso para o Azure Blob Storage.
Envio do deputadosDespesas.parquet concluído
deputados
Leitura do deputados.parquet concluída
Tratamento do deputados.parquet concluído
Arquivo Parquet 'deputados.parquet' enviado com sucesso para o Azure Blob Storage.
Envio do deputados.parquet concluído
